<a href="https://colab.research.google.com/github/xTRam1/Cost-Effective-Reverse-Vending-Machine-Project/blob/main/AI_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Using Modifiable Shell Script
With this step, you can reproduce our results (96% percent accuracy TFlite model) or modify the parameters for your own project.

In [ ]:
# Cloning our GitHub repository
!git clone "https://github.com/xTRam1/Cost-Effective-Reverse-Vending-Machine-Project"

In [ ]:
%cd ai_training
!bash ai_train.sh # You can modify the parameters in this shell script

In [ ]:
# Look at your training summary from Tensorboard
%load_ext tensorboard
%tensorboard --logdir logs/

After training finishes, you must have two new files in your directory - one named "model.tflite" that contains the TensorflowLite model and "labels.txt" that containes a list of the labels in our training dataset

## Training with code (possibly modifying it to your preferences)
This allows for a better visualizing of the model training. It is also better as you can understand each and every component of the Deep Learning process instead of blindly running a line of command. 

### Parameter and dataset initiations
You can modify the parameters below to customize your Deep Learning training. The cell below prepares your data for training. 

In [ ]:
import tensorflow as tf 
import tensorflow_hub as hub
import argparse
import numpy
import datetime

# Parameters to modify
lr = 0.001
batch_size = 32
do_finetuning = True
epoch = 20
# checkpoint_path = "path/to/checkpoint/if/one/exists"
checkpoint_freq = 5

data_root = "path/to/your/dataset"
classifier_model = "https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/4"
datagen_kwargs = dict(rescale=1./255, brightness_range=(0.7, 1.0), vertical_flip=True, horizontal_flip=True, validation_split=0.2) # Pre-processing methods
img_shape = (224, 224)

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
train_data = train_datagen.flow_from_directory(
    data_root,
    shuffle=True,
    target_size=img_shape,
    batch_size=batch_size,
    subset='training'
)
valid_data = train_datagen.flow_from_directory(
    data_root,
    shuffle=True,
    target_size=img_shape,
    batch_size=batch_size,
    subset='validation'
)

### Model initiation
Downloads the pre-trained tensorflow backbone and builds the model. The cell also specifies the loss function and the callback metrics to track during training. 

In [ ]:
model = tf.keras.Sequential([
    # Explicitly define the input shape so the model can be properly
    # loaded by the TFLiteConverter
    tf.keras.layers.InputLayer(input_shape=img_shape+(3,)),
    # Pre-trained model from Tensorflow Hub 
    hub.KerasLayer(classifier_model, trainable=do_finetuning),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(train_data.num_classes, activation='softmax')
])

model.build((None,)+img_shape+(3,))

# If you want to use a previous checkpoint, uncomment the following line
# model.load_weights(args.checkpoint_path)

model.compile(optimizer=tf.keras.optimizers.Adam(lr=lr),
        loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
        metrics=['acc'])
model.summary()
steps_per_epoch = train_data.samples // train_data.batch_size
validation_steps = valid_data.samples // valid_data.batch_size

In [ ]:
# If you want to clear the prior logging data, run this cell
!rm -rf logs

### Training
Below step trains your model and visualizes its training/validation metrics. 

In [ ]:
logdir = "logs/" + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
checkpoint_path = "checkpoints/cp-{epoch:04d}.ckpt"

# Define the basic TensorBoard callback.
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=5*batch_size)

history = model.fit(train_data, epochs=epoch,
    steps_per_epoch=steps_per_epoch,
    validation_data=valid_data,
    validation_steps=validation_steps,
    callbacks=[cp_callback, tensorboard_callback])

In [ ]:
# To see the logs after training, run this command
%tensorboard --logdir logs/

### Testing
Test if your model works correctly before converting it into a TFlite model. 
This cell gives you the validation accuracy and the falsely classified images to diagnose your model and its training parameters.

In [ ]:
import matplotlib.pylab as plt

def get_class_string_from_index(index):
   for class_string, class_index in valid_data.class_indices.items():
      if class_index == index:
         return class_string

error_count = 0
for i in range(valid_data.samples):
    x, y = next(valid_data)
    image = x[0, :, :, :]
    true_index = numpy.argmax(y[0])
    plt.imshow(image)
    plt.axis('off')
    plt.show()

    # Expand the validation image to (1, 224, 224, 3) before predicting the label
    prediction_scores = model.predict(numpy.expand_dims(image, axis=0))
    predicted_index = numpy.argmax(prediction_scores)

    # Look at images that your model is failing to classify correcly to make the necessary adjustments to your model parameters
    if predicted_index != true_index:  
        error_count += 1
        plt.imshow(image)
        plt.axis('off')
        plt.show()
        print("True label: " + get_class_string_from_index(true_index))
        print("Predicted label: " + get_class_string_from_index(predicted_index))

# Prints your model's performance for the validation dataset
print('Validation accuracy is {}'.format((1.0 - (error_count / valid_data.samples)) * 100.0))

### Saving your model and your label list
After the above cell is executed, you must have two new files in your directory - one named "model.tflite" that contains the TensorflowLite model and "labels.txt" that containes a list of the labels in our training dataset

In [ ]:
# Converting model into TensorflowLite model
saved_model_dir = "path/to/save/your/model"
tf.saved_model.save(model, saved_model_dir)

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

# Saving the model and label map into two seperate files for further use
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

labels = '\n'.join(sorted(train_data.class_indices.keys()))

with open('labels.txt', 'w') as f:
    f.write(labels)

# All that is left is to use this TFLite model in your Raspberry Pi 4. 

### Testing your TFLite model
Belove cell tests your TFLite model's performance by printing the model's validation accuracy and by outputting the images which the model has failed classifying correctly. You must use the results of this step to identify any mistakes with your model and adjust the parameters accordingly. 

In [ ]:
import numpy as np
import matplotlib.pylab as plt

def set_input_tensor(interpreter, image):
  tensor_index = interpreter.get_input_details()[0]['index']
  input_tensor = interpreter.tensor(tensor_index)()[0]
  input_tensor[:, :] = image

def load_labels(path):
    """Loads the file path of the labels file."""
    with open(path, 'r') as f:
        return {i: line.strip() for i, line in enumerate(f.readlines())}

# Setting up the model and the labels
labels = load_labels("labels.txt")
interpreter = tf.lite.Interpreter("model_finetune.tflite")

error_count = 0
for _ in range(valid_data.samples):
    interpreter.allocate_tensors()
    _, height, width, _ = interpreter.get_input_details()[0]['shape']
    x, y = next(valid_data)
    image = x[0, :, :, :]
    true_index = np.argmax(y[0])

    top_k = 1
    set_input_tensor(interpreter, image)
    interpreter.invoke()
    output_details = interpreter.get_output_details()[0]
    output = np.squeeze(interpreter.get_tensor(output_details['index']))
    ordered = np.argpartition(-output, top_k)
    results = [(i, output[i]) for i in ordered[:top_k]]
    label_id, prob = results[0]

    if labels[true_index] != labels[label_id]:
        plt.imshow(image)
        plt.axis('off')
        plt.show()
        print("True label: " + labels[true_index] + " Probability: " + str(prob))
        print("Predicted label: " + labels[label_id])
        error_count += 1
print('Validation accuracy is {}'.format((1.0 - (error_count / valid_data.samples)) * 100.0))